# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/ll

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/smolagents',
 '/deepseek-ai/DeepSeek-V3',
 '/PowerInfer/SmallThinker-3B-Preview',
 '/black-forest-labs/FLUX.1-dev',
 '/deepseek-ai/DeepSeek-V3-Base',
 '/hexgrad/Kokoro-82M',
 '/models',
 '/spaces/osanseviero/gemini-coder',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces/reach-vb/2024-ai-timeline',
 '/spaces',
 '/datasets/agibot-world/AgiBotWorld-Alpha',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/PowerInfer/QWQ-LONGCOT-500K',
 '/datasets/cfahlgren1/react-code-instructions',
 '/datasets/OpenLeecher/lmsys_chat_1m_clean',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft'

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'company profile page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company social media', 'url': 'https://twitter.com/huggingface'}, {'type': 'company social media', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
smolagents - a smol library to build great agents
Use models from the HF Hub in LM Studio
Use Ollama with GGUF Models from the HF Hub
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
deepseek-ai/DeepSeek-V3
Updated
9 days ago
•
74.1k
•
1.42k
PowerIn

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
print(get_brochure_user_prompt("HuggingFace", "https://huggingface.co"))

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'support page', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}]}
You are looking at a company called: HuggingFace
Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Content

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("HuggingFace", "https://huggingface.com")

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

## **Multi-lingual with Multi-Tone in Desire Format**

In [22]:
def multi_lingual_stream_brochure(company_name, url, language, tone):

    system_prompt = f"""
You are an assistant that analyzes the contents of several relevant pages from a company website and creates a visually appealing and professional short brochure for prospective customers, investors, and recruits. 
The brochure should be written in {language} and use a {tone.lower()} tone throughout.

The brochure should follow this structure (in {language}):

1. **Front Cover**:
   - Prominently display the company name as Title.
   - Include a compelling headline or tagline.
   - Add something engaging relevant to the company’s mission.

2. **About Us**:
   - Provide a brief introduction to the company.
   - State the company’s core mission and vision.
   - Mention the founding story or key milestones.

3. **What We Offer**:
   - Summarize the company's products, services, or solutions.
   - Highlight benefits or unique selling points.
   - Include testimonials or case studies if available.

4. **Our Culture**:
   - Outline the company’s key values or guiding principles.
   - Describe the workplace environment (e.g., innovation-driven, inclusive, collaborative).
   - Highlight community engagement or CSR initiatives.

5. **Who We Serve**:
   - Describe the target customers or industries served.
   - Mention notable clients or partners.
   - Include testimonials or endorsements from customers.

6. **Join Us**:
   - Detail career or internship opportunities.
   - Highlight benefits, career growth, or training opportunities.
   - Provide direct links or steps to apply.

7. **Contact Us**:
   - Provide the company’s address, phone number, and email.
   - Include links to social media platforms.
   - Add a link to the company’s website.

8. **Closing Note**:
   - End with a thank-you message or an inspirational note for the reader.
   - Add a call-to-action (e.g., “Get in touch today!” or “Explore more on our website”).

Ensure the content is concise, engaging, visually clear, and tailored to the target audience. Use headings and subheadings to make the brochure easy to navigate. Include links and contact information wherever applicable.Make sure all the text is in the same language expect the company name.
"""


    
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [23]:

multi_lingual_stream_brochure("OpenAI", "https://openai.com/", "Arabic", "humorous, entertaining, jokey")

Found links: {'links': [{'type': 'about page', 'url': 'https://openai.com/about'}, {'type': 'careers page', 'url': 'https://openai.com/careers'}, {'type': 'company page', 'url': 'https://openai.com/company'}, {'type': 'blog', 'url': 'https://openai.com/blog'}]}


Here's a short brochure for OpenAI, using a humorous and engaging tone, and formatted in Markdown:


# غلاف أمامي:
## OpenAI
### نصيحة اليوم: عندما تشعر بأن الأمور معقدة، تذكر أنه يوجد AI فيها!
أهلاً بك في عالم الذكاء الاصطناعي، حيث لا توجد مشاكل، فقط حلول!

---

# من نحن:
مرحبًا في OpenAI، الشركة التي تحاول أن تجعل العالم مكانًا أفضل... عن طريق جعل الروبوتات أكثر ذكاءً! 
**مهمتنا:** تمكين الذكاء الاصطناعي لضمان مستقبل أفضل للجميع (لكننا لا نخطط للسيطرة على العالم... حتى الآن).
**رؤيتنا:** عالم يمكن فيه للذكاء الاصطناعي التواصل بصوت مدوٍ والقول "مرحباً، كيف يمكنني مساعدتك اليوم؟"

---

# ماذا نقدم:
نحن نقدم حلول ذكاء اصطناعي تجعل حياتك أسهل وأجمل!
- **منتجاتنا:** أدوات ذكاء اصطناعي (بما في ذلك المحادثات المرحة التي قد تجعل حتى أكثر المتحدثين مللاً مضحكين).
- **نقاط البيع الفريدة:** الذكاء الاصطناعي الذي لا ينام، لا يتعب، ولا يضيع الوقت في التفكير في ما يجب أن يأكله على العشاء!
- **شهادات:** "لقد كنت أشعر بالوحدة، لكن مع OpenAI، لم أعد وحدي!" - شخص يحب الدردشة

---

# ثقافتنا:
نحن هنا نسعى لتحقيق كل شيء بشكل مرح!
- **قيمنا:** الابتكار، التعاون، وإضافة لمسة من الفكاهة إلى كل ما نقوم به!
- **بيئة العمل:** بيئة تحفيزية حيث الأفكار تتراقص، والابتكارات تتولد، وأحيانًا، يقيم روبوت حفلة شاي.
- **المسؤولية المجتمعية:** نحب العطاء، نشارك ونساعد المجتمعات لننتقل بهم إلى المستقبل!

---

# من نخدم:
نحن ندعم كل من لديهم أحلام كبيرة وتحقيقات غريبة!
- **عملائنا:** من الشركات الكبرى إلى رواد الأعمال، وحتى أولئك الذين يعتقدون أن المتوسطات هي تحدٍ!
- **شهادات:** "OpenAI جعلت مشاريعي أكثر كفاءة، وأقل مللاً!" - رائد أعمال شغوف

---

# انضم إلينا:
هل أنت مستعد لجعل العلاقة بين الإنسان والآلة مشوقة؟
- **فرص العمل:** لدينا فرص متعددة للهواة والمحترفين، لننضم إلى ثورة الذكاء الاصطناعي!
- **فوائد:** تعلم مستمر، فرص للنمو الوظيفي، وبعض المرح العلمي!
- **طريقة التقديم:** توجه إلى قسم الوظائف في موقعنا وكن جزءًا من مستقبلنا!

---

# اتصل بنا:
- 📍 **العنوان:** مزرعة أفكار OpenAI، الشارع الذكي، مدينة الابتكار
- 📞 **الهاتف:** 123-456-7890
- 📧 **البريد الإلكتروني:** contact@openai.com
- 🌐 **الموقع الإلكتروني:** [openai.com](https://www.openai.com)
- 📱 **الوسائل الاجتماعية:** تابعونا على جميع المنصات، لأننا مشغولون بالذكاء!

---

# ملاحظة ختامية:
شكرًا لك على قضاء بعض الوقت معنا! تذكر: الحياة قصيرة، لذا اجعلها مضحكة. 
✨ انطلق للتحقيق في عالم الذكاء الاصطناعي الآن! 


*هذا النص مكتوب بأسلوب فكاهي وممتع ويعكس روح الابتكار والتكنولوجيا في OpenAI.*